In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import findspark
findspark.init('/home/rajdeep/spark-3.5.0-bin-hadoop3/')

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [9]:
#initalizing checkpoint path
checkpoint_path = 'checkpoint/'

In [10]:
#initalizing the spark object
spark = SparkSession.builder.appName('wordCount').getOrCreate()

In [ ]:
#create a socket to trasmit data
# in unix shell - nc -lk [port]
# for e.g., nc -lk 9999

In [11]:
#reading the data from console for stream processing
lines = spark.readStream.format("socket").option("host","localhost").option("port","9999").load()

23/12/26 22:06:34 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [12]:
#printing the schema
lines.printSchema()

root
 |-- value: string (nullable = true)



In [13]:
#splitting the input lines into words
word = lines.select(explode(split(col('value'),'\s+')).alias("words"))

In [14]:
#counting the occurence of each words
count = word.groupBy("words").count()

In [15]:
#initalizing stream output details
streamingquery = count.writeStream.format("console").outputMode("complete").trigger(processingTime="1 second").start()

23/12/26 22:06:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2acd4715-0664-496c-a01d-6597827bbc3b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/26 22:06:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
#awaiting the source termination
streamingquery.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
+-----+-----+



23/12/26 22:06:58 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 15921 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
|hello|    1|
+-----+-----+



23/12/26 22:07:07 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 9229 milliseconds
23/12/26 22:07:13 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 6305 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----------+-----+
|     words|count|
+----------+-----+
|      code|    2|
|   runninf|    1|
|  properly|    1|
|     hello|    1|
| streaming|    1|
|        is|    1|
|       the|    1|
|structured|    1|
|   working|    1|
+----------+-----+

